# Feature Engineering for Transformer Failure Prediction

**Purpose:** Create predictive features based on patterns discovered in notebook 02.

**Key insights from previous analysis:**
- Oil temperature spikes 14-21 days before failures
- Temperature differentials (top-bottom) increase before failures
- High sustained load correlates with failures
- Older equipment under stress has higher failure risk

**Goal:** Engineer features, validate predictive power, save for model experiments.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Display settings
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Load Raw Data

In [ ]:
# Load sensor readings
sensor_df = pd.read_csv('../data/raw/sensor_readings.csv', parse_dates=['timestamp'])
print(f"Sensor readings: {sensor_df.shape}")

# Load maintenance log
maint_log = pd.read_csv('../data/raw/maintenance_log.csv', 
                        parse_dates=['install_date', 'last_inspection_date', 'event_date'])
print(f"Maintenance log: {maint_log.shape}")

# Quick check
print("\nSensor columns:", sensor_df.columns.tolist())
print("Maintenance columns:", maint_log.columns.tolist())

## 2. Merge Data Sources

Merge sensor readings with equipment metadata to enable temporal features.

In [ ]:
# Get unique equipment metadata (latest per equipment)
equipment_meta = maint_log.groupby('equipment_id').agg({
    'install_date': 'first',
    'last_inspection_date': 'max'
}).reset_index()

print(f"Equipment metadata records: {len(equipment_meta)}")

# Merge with sensor data
merged_df = sensor_df.merge(equipment_meta, on='equipment_id', how='left')
print(f"Merged data: {merged_df.shape}")

# Calculate temporal features
merged_df['equipment_age_years'] = (merged_df['timestamp'] - merged_df['install_date']).dt.days / 365.25
merged_df['days_since_maintenance'] = (merged_df['timestamp'] - merged_df['last_inspection_date']).dt.days

print(f"\nEquipment age range: {merged_df['equipment_age_years'].min():.1f} - {merged_df['equipment_age_years'].max():.1f} years")
print(f"Days since maintenance range: {merged_df['days_since_maintenance'].min():.0f} - {merged_df['days_since_maintenance'].max():.0f} days")

## 3. Create Target Variables

Label readings based on whether equipment fails within next 30 days.

In [ ]:
# Extract failure events
failures = maint_log[maint_log['event_type'] == 'FAILURE'][['equipment_id', 'event_date']].copy()
print(f"Total failures in dataset: {len(failures)}")

# Initialize target columns
merged_df['failure_30d'] = 0
merged_df['days_to_failure'] = -1

# Label readings preceding failures
for _, failure in failures.iterrows():
    equipment = failure['equipment_id']
    failure_date = failure['event_date']
    
    # Readings within 30 days before failure
    mask = (
        (merged_df['equipment_id'] == equipment) & 
        (merged_df['timestamp'] < failure_date) &
        (merged_df['timestamp'] >= failure_date - pd.Timedelta(days=30))
    )
    
    merged_df.loc[mask, 'failure_30d'] = 1
    merged_df.loc[mask, 'days_to_failure'] = (failure_date - merged_df.loc[mask, 'timestamp']).dt.days

print(f"\nClass distribution:")
print(merged_df['failure_30d'].value_counts())
print(f"\nFailure rate: {merged_df['failure_30d'].mean():.2%}")

## 4. Engineer Features

Based on patterns discovered in notebook 02.

### 4.1 Temperature Features

Oil temperature differentials and trends indicate cooling/insulation issues.

In [ ]:
df = merged_df.copy()

# Temperature differentials (insight from notebook 02)
df['oil_temp_differential'] = df['oil_temp_top_celsius'] - df['oil_temp_bottom_celsius']
df['temp_stress'] = df['winding_temp_celsius'] - df['ambient_temp_celsius']

# Rolling statistics (14-day window per equipment)
df['oil_temp_14d_mean'] = df.groupby('equipment_id')['oil_temp_top_celsius'].transform(
    lambda x: x.rolling(14, min_periods=1).mean()
)
df['oil_temp_14d_std'] = df.groupby('equipment_id')['oil_temp_top_celsius'].transform(
    lambda x: x.rolling(14, min_periods=1).std()
)

# Temperature trend (7-day change)
df['oil_temp_trend'] = df.groupby('equipment_id')['oil_temp_top_celsius'].transform(
    lambda x: x.diff(7)
)

print("Temperature features created:")
print("- oil_temp_differential")
print("- temp_stress")
print("- oil_temp_14d_mean")
print("- oil_temp_14d_std")
print("- oil_temp_trend")

### 4.2 Load Features

High sustained load correlates with failure risk.

In [ ]:
# Rolling load statistics
df['load_14d_max'] = df.groupby('equipment_id')['load_mva'].transform(
    lambda x: x.rolling(14, min_periods=1).max()
)
df['load_14d_mean'] = df.groupby('equipment_id')['load_mva'].transform(
    lambda x: x.rolling(14, min_periods=1).mean()
)

# High load cycles (count of days >75 MVA in last 14 days)
df['high_load_cycles'] = df.groupby('equipment_id')['load_mva'].transform(
    lambda x: (x > 75).rolling(14, min_periods=1).sum()
)

print("Load features created:")
print("- load_14d_max")
print("- load_14d_mean")
print("- high_load_cycles")

### 4.3 Interaction Features

Older equipment under stress = compounding risk.

In [ ]:
# Age × load interaction
df['age_load_interaction'] = df['equipment_age_years'] * df['load_14d_max']

# Temperature × age interaction
df['temp_age_interaction'] = df['oil_temp_14d_mean'] * df['equipment_age_years']

print("Interaction features created:")
print("- age_load_interaction")
print("- temp_age_interaction")

## 5. Feature Validation

Check which features are most predictive of failures.

In [ ]:
# Drop NaN from rolling windows
df_clean = df.dropna()
print(f"Shape after dropping NaN: {df_clean.shape}")

# Feature correlation with target
feature_cols = [
    'oil_temp_differential', 'temp_stress', 'oil_temp_14d_mean', 'oil_temp_14d_std', 'oil_temp_trend',
    'load_14d_max', 'load_14d_mean', 'high_load_cycles',
    'equipment_age_years', 'days_since_maintenance',
    'age_load_interaction', 'temp_age_interaction'
]

correlations = df_clean[feature_cols + ['failure_30d']].corr()['failure_30d'].drop('failure_30d').sort_values(ascending=False)

print("\nFeature correlations with failure_30d:")
print(correlations)

# Visualize
plt.figure(figsize=(10, 6))
correlations.plot(kind='barh')
plt.title('Feature Correlation with Failure (30 days)')
plt.xlabel('Correlation')
plt.tight_layout()
plt.show()

### Quick Random Forest Feature Importance

In [ ]:
# Quick RF to see feature importance
X = df_clean[feature_cols]
y = df_clean['failure_30d']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Train simple RF
rf = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)
rf.fit(X_train, y_train)

# Feature importance
importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 Most Important Features:")
print(importance_df.head(10))

# Visualize
plt.figure(figsize=(10, 6))
importance_df.head(10).plot(x='feature', y='importance', kind='barh')
plt.title('Top 10 Feature Importances (Random Forest)')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()

## 6. Save Features for Model Experiments

Features validated - saving for model experimentation in notebook 04.

In [ ]:
# Save processed features
output_path = '../data/processed/features.csv'
df_clean.to_csv(output_path, index=False)

print(f"Features saved to: {output_path}")
print(f"Shape: {df_clean.shape}")
print(f"Columns: {len(df_clean.columns)}")
print(f"\nFailure rate: {df_clean['failure_30d'].mean():.2%}")
print(f"Total failures: {df_clean['failure_30d'].sum()}")
print(f"Total normal: {(df_clean['failure_30d'] == 0).sum()}")

## Summary

**Feature Engineering Complete**

**Top predictive features:**
1. oil_temp_14d_mean - Rolling average oil temperature
2. age_load_interaction - Older equipment under load
3. temp_stress - Temperature stress indicator
4. load_14d_max - Peak load in recent period
5. high_load_cycles - Sustained high load patterns

**Class imbalance:** ~2-3% failure rate (realistic for well-maintained equipment)

**Next step:** Model experimentation in notebook 04 using features.csv